
# Generative Adversarial Networks (GANs)

Generative Adversarial Networks (GANs) are a class of machine learning frameworks designed to generate new data samples that resemble a given training dataset. They consist of two neural networks, the **generator** and the **discriminator**, which are trained simultaneously in a game-theoretic framework. The generator creates new data samples, while the discriminator evaluates them against real samples from the training set. The goal of the generator is to produce samples that are indistinguishable from real data, while the discriminator aims to correctly classify samples as real or fake. This adversarial process leads to the generator improving its output quality over time, resulting in realistic data generation.

## Example: DCGAN on MNIST

This example demonstrates how to implement a Deep Convolutional GAN (DCGAN) to generate handwritten digits from the MNIST dataset. The DCGAN architecture uses convolutional layers in both the generator and discriminator to effectively learn spatial hierarchies in the data. The generator takes a random noise vector as input and produces a 28x28 grayscale image, while the discriminator takes an image and outputs a probability indicating whether the image is real (from the dataset) or fake (generated by the generator). The training process involves alternating between training the discriminator to distinguish real from fake images and training the generator to produce images that can fool the discriminator. The result is a trained generator capable of producing realistic handwritten digits that resemble those in the MNIST dataset.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

# ───────────────────── 0.  GPU-friendly startup ──────────────────────
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"    # ↓ fragmentation
gpus = tf.config.list_physical_devices("GPU")
print(gpus)
for g in gpus: tf.config.experimental.set_memory_growth(g, True)

In [ ]:
LATENT_DIM, BATCH_SIZE, EPOCHS = 100, 128, 50
SAVE_EVERY, GRID_N = 5, 25

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype("float32") - 127.5) / 127.5    # → [-1, 1]
x_train = np.expand_dims(x_train, -1)
dataset = (tf.data.Dataset.from_tensor_slices(x_train).\
           shuffle(60_000).batch(BATCH_SIZE, drop_remainder=True))

def build_generator():
    return tf.keras.Sequential([
        layers.Input((LATENT_DIM,)),
        layers.Dense(7*7*256, use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(0.2),
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, 5, 1, "same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(64, 5, 2, "same", use_bias=False),
        layers.BatchNormalization(), layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(1, 5, 2, "same", use_bias=False, activation="tanh")
    ], name="generator")

def build_discriminator():
    return tf.keras.Sequential([
        layers.Input((28, 28, 1)),
        layers.Conv2D(64, 5, 2, "same"),  layers.LeakyReLU(0.2), layers.Dropout(0.3),
        layers.Conv2D(128, 5, 2, "same"), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ], name="discriminator")

gen, disc = build_generator(), build_discriminator()

bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_opt = tf.keras.optimizers.Adam(1e-4)
d_opt = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(real_imgs):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    # ---- Discriminator ----
    with tf.GradientTape() as d_tape:
        fake_imgs = gen(noise, training=True)
        real_logits = disc(real_imgs, training=True)
        fake_logits = disc(fake_imgs, training=True)

        d_loss_real = bce(tf.ones_like(real_logits), real_logits)
        d_loss_fake = bce(tf.zeros_like(fake_logits), fake_logits)
        d_loss = (d_loss_real + d_loss_fake) * 0.5

    d_grads = d_tape.gradient(d_loss, disc.trainable_variables)
    d_opt.apply_gradients(zip(d_grads, disc.trainable_variables))

    # ---- Generator ----
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    with tf.GradientTape() as g_tape:
        fake_imgs = gen(noise, training=True)
        fake_logits = disc(fake_imgs, training=True)
        g_loss = bce(tf.ones_like(fake_logits), fake_logits)   # want "real"

    g_grads = g_tape.gradient(g_loss, gen.trainable_variables)
    g_opt.apply_gradients(zip(g_grads, gen.trainable_variables))
    return d_loss, g_loss

os.makedirs("generated", exist_ok=True)
fixed_noise = tf.random.normal([GRID_N, LATENT_DIM])

def save_grid(epoch):
    imgs = gen(fixed_noise, training=False)
    imgs = (imgs * 127.5 + 127.5).numpy().astype("uint8")
    plt.figure(figsize=(5, 5))
    for i in range(GRID_N):
        plt.subplot(5, 5, i+1)
        plt.imshow(imgs[i, :, :, 0], cmap="gray");
        plt.axis("off")
    plt.tight_layout();
    plt.savefig(f"generated/epoch_{epoch:03d}.png");
    plt.close()

for epoch in range(1, EPOCHS+1):
    for real_batch in dataset:
        d_loss, g_loss = train_step(real_batch)
    if epoch == 1 or epoch % SAVE_EVERY == 0:
        print(f"Epoch {epoch:>3}/{EPOCHS}  D: {d_loss:.4f}  G: {g_loss:.4f}")
        save_grid(epoch)

print("Finished training — sample images are in the 'generated/' folder.")